<a href="https://colab.research.google.com/github/pavansai26/umap-vs-clustering/blob/master/difference_between_umap_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
#from sklearn.datasets import fetch_mldata
from sklearn.utils import resample
import time

In [0]:
pip install MulticoreTSNE

In [0]:
from sklearn.manifold import TSNE,LocallyLinearEmbedding,Isomap,MDS,SpectralEmbedding
from sklearn.decomposition import PCA
from MulticoreTSNE import MulticoreTSNE

In [0]:
import umap

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
sns.set(context='notebook',rc={'figure.figsize':(12,10)})

In [0]:
pip install hdbscan

In [0]:
import hdbscan
import sklearn.cluster as cluster

In [0]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

In [0]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [0]:
standard_embedding=umap.UMAP(random_state=42).fit_transform(mnist.data)
plt.scatter(standard_embedding[:,0],standard_embedding[:,1],s=0.1)

TRADITIONAL CLUSTERING

In [0]:
kmeans_labels=cluster.KMeans(n_clusters=10).fit_predict(mnist.data)

In [0]:
plt.scatter(standard_embedding[:,0],standard_embedding[:,1],c=kmeans_labels,s=0.1)

THIS IS NOT really the result we were looking for (though it does expose interesting properties of how k means chooses clusters in high dimensional space and how umap unwraps the manifolds by finding manifold boundaries. while kmeans get some cases correct the two clusters are the far right are mostly correct, most of the rest of the data  looks somewhat arbitarly carved up among the remaining clusters.we can put this impression to the test by evaluating the adjusted rand score and adjust mutual information for this clustering as compared with the true labels.

In [0]:
(adjusted_rand_score(mnist.target,kmeans_labels),adjusted_mutual_info_score(mnist.target,kmeans_labels))

as might expected we have not done a particuraly good job 

as might be expected we have not done a particuraly good job both scores take the values with in the  range from 0 to 1, with 0 representing a bad clustering and 1 representing perfectively recovering the true labels, k means definitely not was arandom , but it was also quite a long way from a perfectly recovering the true labels. part of the problem is way the kmeans works, based on centroids with an assumption of largely spherical clusters. this is rensponsible for some  of the sharp divides that kmeans puts across digit classes.
we can potentially improve on this by using a smarter density based algorithm. in this case we have choosen to try hdbscan, which we believe to be among the most advanced density based techniques. for the sake of performance we will reduce the dimensionality of the data down to 50 dimensions via pca (this recovers the most of the variance) since hdbscan scales somewhat poorly with the dimensionality of the data it will work on.

In [0]:
lowd_mnist=PCA(n_components=50).fit_transform(mnist.data)
hdbscan_labels=hdbscan.HDBSCAN(min_samples=10,min_cluster_size=500).fit_predict(lowd_mnist)

we can now inspect the results. before we do however, it should be noted that one of the features of hdbscan is that it can refuse to cluster some points and classify the as noise to visualize this aspect we will colorprints that we are classifed as noise gray and then color the remaining points according to the cluster membership.

In [0]:
clustered=(hdbscan_labels >= 0)
plt.scatter(standard_embedding[clustered,0],standard_embedding[clustered,1],
            c=hdbscan_labels[clustered],s=0.1)
plt.scatter(standard_embedding[clustered,0],standard_embedding[clustered,1],
            c=(0.5,0.5,0.5),s=0.1)

In [0]:
(adjusted_rand_score(mnist.target,hdbscan_labels),adjusted_mutual_info_score(mnist.target,hdbscan_labels))

these scores are far worse than kmeans partially this is due to the fact that these scores assume that the noise points are simply an extra  cluster. we can instead only look at the data that hdbscan was actually confident enough to assign to clusters  a simple sub selection will let us recompute the scores for only that data

In [0]:
clustered=(hdbscan_labels >=0)
(adjusted_rand_score(mnist.target[clustered],hdbscan_labels[clustered]),
 adjusted_mutual_info_score(mnist.target[clustered],hdbscan_labels[clustered]))

In [0]:
np.sum(clustered)/mnist.data.shape[0]